In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### PREPERATIONS

In [13]:
file_name = 'hair_problem.csv' # Pas dit pad aan!

try:
    df = pd.read_csv(file_name)
except FileNotFoundError:
    print(f"Fout: Bestand '{file_name}' niet gevonden. Pas het pad aan.")
    exit()

In [19]:
df.rename(columns={
    "Timestamp": "timestamp",
    "What is your name ?": "name",
    "What is your age ?": "age",
    "What is your gender ?": "gender",
    "Do you have hair fall problem ?": "hair_fall",
    "Is there anyone in your family having a hair fall problem or a baldness issue?": "family_history",
    "Did you face any type of chronic illness in the past?": "chronic_illness",
    "Do you stay up late at night?": "late_night",
    "Do you have any type of sleep disturbance?": "sleep_disturbance",
    "Do you think that in your area water is a reason behind hair fall problems?": "water_reason",
    "Do you use chemicals, hair gel, or color in your hair?": "hair_chemicals",
    "Do you have anemia?": "anemia",
    "Do you have too much stress": "stress",
    "What is your food habit": "food_habit"
}, inplace=True)
df.head()

,age,gender,hair_fall,family_history,chronic_illness,late_night,sleep_disturbance,water_reason,hair_chemicals,anemia,stress,food_habit
0,22,Male,Yes,Yes,Yes,Yes,No,No,No,Yes,Yes,Both
1,18,Male,No,No,No,No,No,Yes,Yes,No,No,Both
2,25,Male,Yes,Yes,Yes,No,No,Yes,No,No,Yes,Both
3,17,Male,No,No,No,Yes,No,Yes,No,Yes,Yes,Both
4,28,Male,No,Yes,Yes,Yes,No,Yes,No,Yes,Yes,Nutritious


In [20]:
target = ['hair_fall']
features = ['stress', 'water_reason', 'sleep_disturbance', 'hair_chemicals', 'family_history']

X = df[features]
y = df[target]

print(X.head())
print(y.head())

  stress water_reason sleep_disturbance hair_chemicals family_history
0    Yes           No                No             No            Yes
1     No          Yes                No            Yes             No
2    Yes          Yes                No             No            Yes
3    Yes          Yes                No             No             No
4    Yes          Yes                No             No            Yes
  hair_fall
0       Yes
1        No
2       Yes
3        No
4        No


In [21]:
def map_yes_no(value):
    return 1 if value == 'Yes' else 0

for col in features:
    if col in X.columns:
        X.loc[:, col] = X[col].apply(map_yes_no).astype(np.float32) # Gebruik float32 voor TF
y = df['hair_fall'].apply(map_yes_no).astype(np.float32)


In [22]:
print(X.head())
print(y.head())

  stress water_reason sleep_disturbance hair_chemicals family_history
0    1.0          0.0               0.0            0.0            1.0
1    0.0          1.0               0.0            1.0            0.0
2    1.0          1.0               0.0            0.0            1.0
3    1.0          1.0               0.0            0.0            0.0
4    1.0          1.0               0.0            0.0            1.0
0    1.0
1    0.0
2    1.0
3    0.0
4    0.0
Name: hair_fall, dtype: float32


In [23]:
# Splits de data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Normaliseer de features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)   
# Converteer naar numpy arrays
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)
X_test_np = np.array(X_test)
y_test_np = np.array(y_test)
print("✅ Data Voorbereid.")

✅ Data Voorbereid.


### Model

In [24]:
num_features = len(features)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(num_features,)),
    tf.keras.layers.Dense(1, activation='sigmoid', name='logistic_output')
])

# Logistische Regressie gebruikt Binary Crossentropy
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\nKeras Model Samenvatting:")
model.summary()


Keras Model Samenvatting:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 logistic_output (Dense)     (None, 1)                 6         
                                                                 
Total params: 6 (24.00 Byte)
Trainable params: 6 (24.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
print("\nTraining Keras Model...")
model.fit(
    X_train_np,
    y_train_np,
    epochs=50,
    batch_size=32,
    verbose=0
)
print("✅ Keras Model Getraind.")
model.save('hair_fall_model_tf.keras')


Training Keras Model...
✅ Keras Model Getraind.


In [26]:
import tf2onnx
import onnx
import tensorflow as tf

model = tf.keras.models.load_model('hair_fall_model_tf.keras')

onnx_model, _ = tf2onnx.convert.from_keras(model)
onnx.save(onnx_model, 'hair_fall_model.onnx')

print("✅ Model geconverteerd naar ONNX-formaat.")

Could not search for non-variable resources. Concrete function internal representation may have changed.


✅ Model geconverteerd naar ONNX-formaat.


In [ ]:
weights = model.get_weights()
print("\nModel Weights:")
for i, w in enumerate(weights):
    print(f"Layer {i} weights:")
    print(w)

weights_data = {
    'coefficients': weights[0].flatten().tolist(),
    'bias': float(weights[1][0]),
    'feature_names': features
}

with open('model_weights.json', 'w') as f:
    import json
    json.dump(weights_data, f, indent=2)    


Model Weights:
Layer 0 weights:
[[0.773751  ]
 [0.23789515]
 [0.3901703 ]
 [0.6359475 ]
 [0.64180905]]
Layer 1 weights:
[0.611417]


In [2]:
# import tf2onnx
# import onnx
# import tensorflow as tf
# import numpy as np

# print("TensorFlow:", tf.__version__)
# print("tf2onnx:", tf2onnx.__version__)
# print("NumPy:", np.__version__)

TensorFlow: 2.13.0
tf2onnx: 1.15.1
NumPy: 1.24.3
